In [2]:
import numpy as np
import math
from scipy.stats import norm
from datetime import date, timedelta

def black_scholes_formula(option_type, underlying_price, strike_price, time_to_maturity, volatility):
    if time_to_maturity <= 0:
      time_to_maturity = 0.000001
    risk_free_rate = 0.0525
    d1 = (math.log(underlying_price / strike_price) + (risk_free_rate + (volatility**2) / 2) * time_to_maturity) / (volatility * math.sqrt(time_to_maturity))
    d2 = d1 - volatility * math.sqrt(time_to_maturity)

    if option_type == 'call':
        call_option_price = underlying_price * norm.cdf(d1) - strike_price * math.exp(-risk_free_rate * time_to_maturity) * norm.cdf(d2)
        return call_option_price
    elif option_type == 'put':
        put_option_price = strike_price * math.exp(-risk_free_rate * time_to_maturity) * norm.cdf(-d2) - underlying_price * norm.cdf(-d1)
        return put_option_price
    else:
        return None  # Invalid option type

def getCall(spot,strike,dte,sigma):
    return black_scholes_formula("call",spot, strike, dte/365, sigma)

def getPut(spot,strike,dte,sigma):
    return black_scholes_formula("put",spot, strike, dte/365, sigma)

In [3]:
# Underlying price (per share): S;
# Strike price of the option (per share): K;
# Time to maturity (years): T;
# Continuously compounding risk-free interest rate: r;
# Continuously compounding dividend: q;
# Volatility: sigma;

# Implied Volatility using bisection
# quantconnect.com/forum/discussion/2269/generate-volatility-surface-plot-by-interpolation/p1
def implied_vol(option_type, option_price, S, K, r, T, q=0):
    # apply bisection method to get the implied volatility by solving the BSM function
    precision = 0.00001
    upper_vol = 50.0
    max_vol = 50.0
    min_vol = 0.0001
    lower_vol = 0.0001
    iteration = 0

    while 1:
        iteration +=1
        mid_vol = (upper_vol + lower_vol)/2.0
        #bs_call(S,K,T,r,q,sigma):
        #getCall(spot,strike,dte,sigma):
        if option_type == 'c':
            price = getCall(S,K,T,mid_vol)
            lower_price = getCall(S,K,T,lower_vol)
            if (lower_price - option_price) * (price - option_price) > 0:
                lower_vol = mid_vol
            else:
                upper_vol = mid_vol
            if abs(price - option_price) < precision: break
            if mid_vol > max_vol - 5 :
                mid_vol = 0.0001
                break

        elif option_type == 'p':
            price = getPut(S,K,T,mid_vol)
            upper_price = getPut(S,K,T,upper_vol)

            if (upper_price - option_price) * (price - option_price) > 0:
                upper_vol = mid_vol
            else:
                lower_vol = mid_vol

            if abs(price - option_price) < precision: break

            if iteration > 100: break

    return mid_vol

In [5]:
#usar aqui getCall y getPut
#ejemplo con datos del SPY en optionstrat
#bs_call(191.94,190,28/365,0.0525,0,0.2317) 6.3490

spot = 445.36
strike1 = 450
d0 = date.today()
d1 = date(2024, 12, 20)
delta = (d1 - d0).days
print(delta)
dte = delta #days or hours/24:
iv1 = 0.132

getCall(spot, strike1, dte, iv1)

22


np.float64(4.311286762924823)

In [6]:
strike1 = 420
d0 = date.today()
d1 = date(2024, 12, 20)
delta = (d1 - d0).days
dte = delta
iv1 = 0.168
prima = getCall(spot, strike1, dte, iv1)
print(f'dte: {delta}, prima: {prima}')

dte: 22, prima: 27.210993960661597


In [7]:
spot = 455
strike1 = 450
dte = 5
iv1 = 0.157
prima = getCall(spot, strike1, dte, iv1)
print(f'dte: {dte}, prima: {prima}')

dte: 5, prima: 6.635441012108004


In [ ]:
# call/put option price, underlying price, strike price, time to maturity, risk free rate, implied volatility
# Underlying price (per share): S;
# Strike price of the option (per share): K;
# Time to maturity (years): T;
# Continuously compounding risk-free interest rate: r;
# Continuously compounding dividend: q;
# Volatility: sigma;
# def implied_vol(option_type, option_price, S, K, r, T, q=0):
#          option_type, price, Spot, Strike, Free Rate, DTE
print (implied_vol('c', 6.3490, 191.94, 190, 0.0525, 28))
print (implied_vol('p', 3.90, 191.94, 190, 0.0525, 28))